In [1]:
"""
4 exp 4 temps

self dist
self dist shifted
mlp vanilla
cnn vanilla
cnn mlp 
cnn stupider + mlp
fidelity of mlp to t' cross fidelity wrt first cnn - 1 plot

accuracy NLL ECE 

top1 agreement between teacher and student,
KL divergence, invariance metric (crossentropy?) -> show patrick it's better

FLOPS!!
"""

"\n4 exp 4 temps\n\nself dist\nself dist shifted\nmlp vanilla\ncnn vanilla\ncnn mlp \ncnn stupider + mlp\nfidelity of mlp to t' cross fidelity wrt first cnn - 1 plot\n\naccuracy NLL ECE \n\ntop1 agreement between teacher and student,\nKL divergence, invariance metric (crossentropy?) -> show patrick it's better\n\nFLOPS!!\n"

In [2]:
#imports
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from models.cnn import SimpleCNN
from models.resnet import resnet18_mnist
from models.mlp import MLP
from distillation_utils import Distiller
from invariances_utils import shift_preserving_shape, test_IM, validate
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

In [3]:
torch.cuda.empty_cache()
print(torch.cuda.memory_allocated())
print(torch.cuda.memory_cached())

0
0


C:\Users\nicol\AppData\Roaming\Python\Python310\site-packages\torch\cuda\memory.py:444: FutureWarning: torch.cuda.memory_cached has been renamed to torch.cuda.memory_reserved
  warnings.warn(


In [4]:
temperature = 1
#random_seed = 42

In [5]:
in_channels = 1
num_epochs = 10
num_classes = 10
batch_size = 64
lr = 0.001
TRAIN = False
device = 'cuda'

In [6]:
# MNIST dataset

# Define a custom dataset that combines MNIST and additional data
class ShiftAugmentedMNIST(Dataset):
    def __init__(self, mnist_dataset, translation_times : int = 5, max_shift : int = 5):
        self.mnist_dataset = mnist_dataset
        directions = ["u","d","l","r"]
        self.translations = []
        for i in range(len(self.mnist_dataset)):
            img, label = self.mnist_dataset[i]
            img = img.squeeze()
            for t in range(translation_times):
                sh = shift_preserving_shape(img, direction=directions[np.random.randint(0,4)],
                                            max_shift=max_shift).unsqueeze(0)
                if sh is not None:
                    self.translations.append((sh, label))

    def __getitem__(self, index):
        if index < len(self.mnist_dataset):
            return self.mnist_dataset[index]
        else:
            return self.translations[index - len(self.mnist_dataset)]

    def __len__(self):
        return len(self.mnist_dataset) + len(self.translations)


transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transform, download=True)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

train_augmented_dataset = ShiftAugmentedMNIST(train_dataset)
train_augmented_loader = DataLoader(dataset=train_augmented_dataset, batch_size=batch_size, shuffle=True)

Undistilled MLP

In [5]:
#Loading undistilled MLP
if TRAIN:
    undistilled_mlp = MLP(input_dim = 784, output_dim= num_classes, hidden_size= 2048,
        hidden_layers= 4, device='cuda')
    criterion_mlp = torch.nn.CrossEntropyLoss()
    optimizer_mlp = torch.optim.Adam(undistilled_mlp.parameters(), lr=lr)
    undistilled_mlp.training_loop(train_loader=train_loader, optimizer=optimizer_mlp, criterion=criterion_mlp, 
              num_epochs=5, save_path_folder="saved_models_undistilled")
if not TRAIN:
    undistilled_mlp = MLP(input_dim = 784, output_dim= num_classes, hidden_size= 2048,
            hidden_layers= 4, device='cuda', from_saved_state_dict="saved_models_undistilled/mlp")

validate(model=undistilled_mlp, weights_file=None, valid_data=test_loader, device=device, is_mlp= True)

Not using softmax
Correct normal: 0.9714
Correct shifted: 0.3009

accu: 0.97153664

nlll: 0.10431392

ecel: 0.03115511

test_IM: 0.89037299



{'accu': 0.9715366363525391,
 'nlll': 0.1043139174580574,
 'ecel': 0.031155109405517578,
 'test_IM': tensor(0.8904, device='cuda:0')}

Undistilled MLP Augmented

In [6]:
#Loading undistilled MLP augmented
if TRAIN:
    undistilled_mlp_augmented = MLP(input_dim = 784, output_dim= num_classes, hidden_size= 2048,
        hidden_layers= 4, device='cuda')
    criterion_mlp = torch.nn.CrossEntropyLoss()
    optimizer_mlp = torch.optim.Adam(undistilled_mlp_augmented.parameters(), lr=lr)
    undistilled_mlp_augmented.training_loop(train_loader=train_augmented_loader, optimizer=optimizer_mlp, criterion=criterion_mlp, 
              num_epochs=5, save_path_folder="saved_models_undistilled_augmented")
if not TRAIN:
    undistilled_mlp_augmented = MLP(input_dim = 784, output_dim= num_classes, hidden_size= 2048,
            hidden_layers= 4, device='cuda', from_saved_state_dict="saved_models_undistilled_augmented/mlp")

validate(model=undistilled_mlp_augmented, weights_file="saved_models_undistilled_augmented/mlp", valid_data=test_loader, device=device, is_mlp= True)

Not using softmax
Correct normal: 0.9691
Correct shifted: 0.9673

accu: 0.96924764

nlll: 0.11307620

ecel: 0.03176201

test_IM: 0.06156954



{'accu': 0.9692476391792297,
 'nlll': 0.11307619512081146,
 'ecel': 0.0317620113492012,
 'test_IM': tensor(0.0616, device='cuda:0')}

CNN_1

In [7]:
#Obtaining CNN
cnn_path = "saved_cnn/model_1"
cnn = resnet18_mnist().to(device)
if TRAIN:
    criterion_cnn = torch.nn.CrossEntropyLoss()
    optimizer_cnn = torch.optim.Adam(cnn.parameters(), lr=lr)
    # model training
    for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(train_loader):
            outputs = cnn(images.to('cuda'))
            loss = criterion_cnn(outputs, labels.to('cuda'))

            optimizer_cnn.zero_grad()
            loss.backward()
            optimizer_cnn.step()

            if (i + 1) % 100 == 0:
                print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')
    # Save the trained model
    torch.save(cnn.state_dict(), cnn_path)
    print(f"Model saved as {cnn_path}!")
if not TRAIN:
    state_dict = torch.load(cnn_path)
    cnn.load_state_dict(state_dict=state_dict)

validate(model=cnn, weights_file=None, valid_data=test_loader, device=device, is_mlp= False)

Loading Resnet18-aps model for MNIST
Correct normal: 0.9895
Correct shifted: 0.9871

accu: 0.98955017

nlll: 0.03326803

ecel: 0.01363378

test_IM: 0.00982955



{'accu': 0.9895501732826233,
 'nlll': 0.0332680344581604,
 'ecel': 0.01363377831876278,
 'test_IM': tensor(0.0098, device='cuda:0')}

CNN_2

In [10]:
# TODO: create second "stupid" cnn model (at the moment it is the same)
#Obtaining CNN
cnn_path = "saved_cnn/model_2"
#cnn = SimpleCNN(in_channels=in_channels, num_classes=num_classes, num_conv_layers=num_conv_layers, temperature=temperature).to('cuda:0')
num_epochs = 5
cnn = resnet18_mnist().to(device)
if TRAIN:
    criterion_cnn = torch.nn.CrossEntropyLoss()
    optimizer_cnn = torch.optim.Adam(cnn.parameters(), lr=lr)
    # model training
    for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(train_loader):
            outputs = cnn(images.to('cuda'))
            loss = criterion_cnn(outputs, labels.to('cuda'))

            optimizer_cnn.zero_grad()
            loss.backward()
            optimizer_cnn.step()

            if (i + 1) % 100 == 0:
                print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')
    # Save the trained model
    torch.save(cnn.state_dict(), cnn_path)
    print(f"Model saved as {cnn_path}!")
if not TRAIN:
    state_dict = torch.load(cnn_path)
    cnn.load_state_dict(state_dict=state_dict)

validate(model=cnn, weights_file=None, valid_data=test_loader, device=device, is_mlp= False)


Loading Resnet18-aps model for MNIST
Correct normal: 0.9911
Correct shifted: 0.9907

accu: 0.99114251

nlll: 0.02678524

ecel: 0.01055782

test_IM: 0.00580762



{'accu': 0.9911425113677979,
 'nlll': 0.026785239577293396,
 'ecel': 0.010557817295193672,
 'test_IM': tensor(0.0058, device='cuda:0')}

Self distilling MLP 

In [7]:
#Self distilling MLP (only from unshifted data)
save_path_folder = "saved_models_selfdistill_unshifted/"
teacher_path = "saved_models_undistilled/mlp"
mlp_student = MLP(input_dim = 784, output_dim= num_classes, hidden_size= 2048,
            hidden_layers= 4, device=device)

mlp_teacher = MLP(input_dim = 784, output_dim= num_classes, hidden_size= 2048,
            hidden_layers= 4, device=device, from_saved_state_dict=teacher_path)

validate(model=mlp_teacher, weights_file=None, valid_data=test_loader, device=device, is_mlp= True)

if TRAIN:
    selfdistiller = Distiller(student=mlp_student, teacher=mlp_teacher, device=device, lr=0.001, is_teacher_mlp=True)
    selfdistiller.distill(train_data=train_loader, valid_data=test_loader, save_path_folder= save_path_folder) # TODO the model will not be saved in distill() or is it inherited?? put code for saving student
if not TRAIN:
    print("Loading params")
    selfdistiller = Distiller(student=mlp_student, teacher=mlp_teacher, device=device, lr=0.001,
                        load_student_from_path = save_path_folder + 'distiller', is_teacher_mlp=True)
selfdistiller.compute_fidelity(test_loader)
validate(model=selfdistiller.get_student(), weights_file=None, valid_data=test_loader, device=device, is_mlp= True)

Not using softmax
Not using softmax
Correct normal: 0.9714
Correct shifted: 0.2996

accu: 0.97153664

nlll: 0.10431392

ecel: 0.03115512

test_IM: 0.89163655

Loading params
Correct normal: 0.9355
Correct shifted: 0.2326

accu: 0.93550956

nlll: 0.37540725

ecel: 0.05995478

test_IM: 1.00565004



{'accu': 0.9355095624923706,
 'nlll': 0.37540724873542786,
 'ecel': 0.05995477735996246,
 'test_IM': tensor(1.0057, device='cuda:0')}

Self distilling MLP Augmented

In [6]:
#Self distilling MLP (augmented)
save_path_folder = "saved_models_selfdistill_augmented"
teacher_path = "saved_models_undistilled_augmented/mlp"
mlp_student = MLP(input_dim = 784, output_dim= num_classes, hidden_size= 2048,
            hidden_layers= 4, device=device)

mlp_teacher = MLP(input_dim = 784, output_dim= num_classes, hidden_size= 2048,
            hidden_layers= 4, device=device, from_saved_state_dict=teacher_path)
validate(model=mlp_teacher, weights_file=teacher_path, valid_data=test_loader, device=device, is_mlp= True)

if TRAIN:
    selfdistiller = Distiller(student=mlp_student, teacher=mlp_teacher, device=device, lr=0.001, is_teacher_mlp=True)
    selfdistiller.distill(train_data=train_loader, valid_data=test_loader, save_path_folder= save_path_folder) # TODO the model is not saved in distill() or is it inherited??
    # selfdistiller.test_step(test_loader=test_loader) TODO: there is not test_step method?
if not TRAIN:
    print("Loading params")
    selfdistiller = Distiller(student=mlp_student, teacher=mlp_teacher, device=device, lr=0.001,
                        load_student_from_path = save_path_folder + 'distiller', is_teacher_mlp=True)
    # selfdistiller.test_step(test_loader=test_loader) TODO: there is not test_step method?
selfdistiller.compute_fidelity(test_loader)
validate(model=mlp_student, weights_file=None, valid_data=test_loader, device=device, is_mlp= True)

Not using softmax
Not using softmax
Correct normal: 0.9691
Correct shifted: 0.9661

accu: 0.96924764

nlll: 0.11307620

ecel: 0.03176200

test_IM: 0.06044262

Loading params
Correct normal: 0.9653
Correct shifted: 0.3377

accu: 0.96546578

nlll: 0.15674105

ecel: 0.03404006

test_IM: 0.87618351



{'accu': 0.965465784072876,
 'nlll': 0.15674105286598206,
 'ecel': 0.03404005616903305,
 'test_IM': tensor(0.8762, device='cuda:0')}

Distilling CNN_1 -> MLP_1 

Distilling CNN_2 -> MLP_2

In [7]:
#Distilling CNN to MLP
TRAIN = True
save_path_folder_1 = "saved_models_distill_cnn1_to_mlp1"
save_path_folder_2 = "saved_models_distill_cnn2_to_mlp2"
teacher_path_1 = "saved_cnn/model_1"
teacher_path_2 = "saved_cnn/model_2"

mlp_student_1 = MLP(input_dim = 784, output_dim= num_classes, hidden_size= 2048,
            hidden_layers= 4, device=device)

mlp_student_2 = MLP(input_dim = 784, output_dim= num_classes, hidden_size= 2048,
            hidden_layers= 4, device=device)

teacher_1 = resnet18_mnist().to(device)
state_dict = torch.load(teacher_path_1)
teacher_1.load_state_dict(state_dict=state_dict)
validate(model=teacher_1, weights_file=None, valid_data=test_loader, device=device, is_mlp= False)

#TODO: use different architecture than mlp_teacher_1 (at the moment it is the same)
teacher_2 = resnet18_mnist().to(device)
state_dict = torch.load(teacher_path_2)
teacher_2.load_state_dict(state_dict=state_dict)
validate(model=teacher_2, weights_file=None, valid_data=test_loader, device=device, is_mlp= False)

if TRAIN:
    student1_teacher1 = Distiller(student=mlp_student_1, teacher=teacher_1, device=device, lr=0.001)
    student1_teacher1.distill(train_data=train_loader, valid_data=test_loader, save_path_folder= save_path_folder_1) # TODO the model is not saved in distill() or is it inherited??
    student1_teacher1.compute_fidelity(test_loader)

    student2_teacher2 = Distiller(student=mlp_student_2, teacher=teacher_2, device=device, lr=0.001)
    student2_teacher2.distill(train_data=train_loader, valid_data=test_loader, save_path_folder= save_path_folder_2) # TODO the model is not saved in distill() or is it inherited??
    student2_teacher2.compute_fidelity(test_loader)

if not TRAIN:
    print("Loading params")
    student1_teacher1 = Distiller(student=mlp_student_1, teacher=teacher_1, device=device, lr=0.001,
                        load_student_from_path = save_path_folder_1 + 'distiller')
    student1_teacher1.compute_fidelity(test_loader)

    student2_teacher2 = Distiller(student=mlp_student_2, teacher=teacher_2, device=device, lr=0.001,
                        load_student_from_path = save_path_folder_2 + 'distiller')
    student2_teacher2.compute_fidelity(test_loader)

validate(model=student1_teacher1.get_student(), weights_file=None, valid_data=test_loader, device=device, is_mlp= True)
validate(model=student2_teacher2.get_student(), weights_file=None, valid_data=test_loader, device=device, is_mlp= True)
student2_teacher1 = Distiller(student=student2_teacher2.get_student(), teacher=teacher_1, device=device, lr=0.001)
validate(model=student2_teacher1.get_student(), weights_file=None, valid_data=test_loader, device=device, is_mlp= True)
student2_teacher1.compute_fidelity(test_loader)


Not using softmax
Not using softmax
Loading Resnet18-aps model for MNIST
Correct normal: 0.9895
Correct shifted: 0.986

accu: 0.98955017

nlll: 0.03326803

ecel: 0.01363377

test_IM: 0.01049326

Loading Resnet18-aps model for MNIST
Correct normal: 0.9911
Correct shifted: 0.9896

accu: 0.99114251

nlll: 0.02678524

ecel: 0.01055783

test_IM: 0.00580127


Distilling...
Epoch : 1/10
Student train loss = 0.80264387
Distill train loss = 4.04525003
Student train accu = 0.84331690

Student valid loss = 0.37215309
Distill valid loss = 1.74706853
Student valid accu = 0.93242436
---
Epoch : 2/10
Student train loss = 0.32149671
Distill train loss = 1.37546878
Student train accu = 0.94879398

Student valid loss = 0.23742334
Distill valid loss = 1.10630628
Student valid accu = 0.95650876
---
Epoch : 3/10
Student train loss = 0.23480324
Distill train loss = 1.06259051
Student train accu = 0.95922175

Student valid loss = 0.20575929
Distill valid loss = 0.85405749
Student valid accu = 0.96357484
---


{'top1_agreement': 0.976015127388535, 'teach_stu_kldiv': 0.138189376044535}

validate(model=student1_teacher1.get_student(), weights_file=None, valid_data=test_loader, device=device, is_mlp= True)

Correct normal: 0.9725
Correct shifted: 0.3355

accu: 0.97263139

nlll: 0.14249569

ecel: 0.02575893

test_IM: 0.89621401

-----------------------------------------
validate(model=student2_teacher2.get_student(), weights_file=None, valid_data=test_loader, device=device, is_mlp= True)

Correct normal: 0.9784
Correct shifted: 0.3627

accu: 0.97850323

nlll: 0.10672864

ecel: 0.02120134

test_IM: 0.86981630

--------------------------------------------

student1_teacher1.compute_fidelity(test_loader)
{'top1_agreement': 0.9727308917197452, 'teach_stu_kldiv': 0.1490013027836841}

student2_teacher2.compute_fidelity(test_loader)
{'top1_agreement': 0.9791003184713376, 'teach_stu_kldiv': 0.09948632423916574}

student2_teacher1.compute_fidelity(test_loader)
{'top1_agreement': 0.976015127388535, 'teach_stu_kldiv': 0.138189376044535}

In [11]:
#Distilling CNN to MLP
TRAIN = False
save_path_folder_1 = "saved_models_distill_cnn1_to_mlp1"
save_path_folder_2 = "saved_models_distill_cnn2_to_mlp2"
teacher_path_1 = "saved_cnn/model_1"
teacher_path_2 = "saved_cnn/model_2"

mlp_student_1 = MLP(input_dim = 784, output_dim= num_classes, hidden_size= 2048,
            hidden_layers= 4, device=device)

mlp_student_2 = MLP(input_dim = 784, output_dim= num_classes, hidden_size= 2048,
            hidden_layers= 4, device=device)

teacher_1 = resnet18_mnist().to(device)
state_dict = torch.load(teacher_path_1)
teacher_1.load_state_dict(state_dict=state_dict)
validate(model=teacher_1, weights_file=None, valid_data=test_loader, device=device, is_mlp= False)

#TODO: use different architecture than mlp_teacher_1 (at the moment it is the same)
teacher_2 = resnet18_mnist().to(device)
state_dict = torch.load(teacher_path_2)
teacher_2.load_state_dict(state_dict=state_dict)
validate(model=teacher_2, weights_file=None, valid_data=test_loader, device=device, is_mlp= False)

if TRAIN:
    student1_teacher1 = Distiller(student=mlp_student_1, teacher=teacher_1, device=device, lr=0.001)
    student1_teacher1.distill(train_data=train_loader, valid_data=test_loader, save_path_folder= save_path_folder_1) # TODO the model is not saved in distill() or is it inherited??
    student1_teacher1.compute_fidelity(test_loader)

    student2_teacher2 = Distiller(student=mlp_student_2, teacher=teacher_2, device=device, lr=0.001)
    student2_teacher2.distill(train_data=train_loader, valid_data=test_loader, save_path_folder= save_path_folder_2) # TODO the model is not saved in distill() or is it inherited??
    student2_teacher2.compute_fidelity(test_loader)

if not TRAIN:
    print("Loading params")
    student1_teacher1 = Distiller(student=mlp_student_1, teacher=teacher_1, device=device, lr=0.001,
                        load_student_from_path = save_path_folder_1 + 'distiller')
    student1_teacher1.compute_fidelity(test_loader)

    student2_teacher2 = Distiller(student=mlp_student_2, teacher=teacher_2, device=device, lr=0.001,
                        load_student_from_path = save_path_folder_2 + 'distiller')
    student2_teacher2.compute_fidelity(test_loader)

validate(model=student1_teacher1.get_student(), weights_file=None, valid_data=test_loader, device=device, is_mlp= True)
validate(model=student2_teacher2.get_student(), weights_file=None, valid_data=test_loader, device=device, is_mlp= True)
student2_teacher1 = Distiller(student=student2_teacher2.get_student(), teacher=teacher_1, device=device, lr=0.001)
validate(model=student2_teacher1.get_student(), weights_file=None, valid_data=test_loader, device=device, is_mlp= True)
student2_teacher1.compute_fidelity(test_loader)

Not using softmax
Not using softmax
Loading Resnet18-aps model for MNIST
Correct normal: 0.9895
Correct shifted: 0.9865

accu: 0.98955017

nlll: 0.03326803

ecel: 0.01363378

test_IM: 0.00957882

Loading Resnet18-aps model for MNIST
Correct normal: 0.9911
Correct shifted: 0.9907

accu: 0.99114251

nlll: 0.02678524

ecel: 0.01055783

test_IM: 0.00578883

Loading params
Correct normal: 0.9725
Correct shifted: 0.3381

accu: 0.97263139

nlll: 0.14249569

ecel: 0.02575892

test_IM: 0.89266229

Correct normal: 0.9784
Correct shifted: 0.3579

accu: 0.97850323

nlll: 0.10672864

ecel: 0.02120135

test_IM: 0.87484848

Correct normal: 0.9784
Correct shifted: 0.3606

accu: 0.97850323

nlll: 0.10672864

ecel: 0.02120134

test_IM: 0.86885935



{'top1_agreement': 0.976015127388535, 'teach_stu_kldiv': 0.138189376044535}